In [1]:
import numpy as np


In [2]:
def parse_input(filename: str):
    elf_lst = []
    with open(filename) as f:
        for row_idx, row in enumerate(f):
            row = row.rstrip()
            for col_idx, char in enumerate(row):
                if char == "#":
                    elf_lst.append((col_idx, row_idx))
        
    return np.array(elf_lst, dtype=int)

In [3]:
from tqdm import tqdm, trange
diffs = np.stack([
    np.array([-1, -1]),
    np.array([0, -1]),
    np.array([1, -1]),
    np.array([1, 0]),
    np.array([1, 1]),
    np.array([0, 1]),
    np.array([-1, 1]),
    np.array([-1, 0]),
    np.array([-1, -1]),
])

def print_field(elf_pos: np.ndarray):
    elf_pos_set = set(map(tuple, elf_pos))
    ret_str = ""
    for row_idx in range(elf_pos[:, 1].min(), elf_pos[:, 1].max()+1):
        for col_idx in range(elf_pos[:, 0].min(), elf_pos[:, 0].max()+1):
            if (col_idx, row_idx) in elf_pos_set:
                ret_str += "#"
            else:
                ret_str += "."
        ret_str += "\n"
    print(ret_str)


def task_one(elf_pos: set[np.ndarray], num_rounds: int):
    dirs = np.array([0, 2, 3, 1], dtype=int)
    for _round in trange(num_rounds):
        proposed_moves = []
        for elf_idx, elf in enumerate(elf_pos):
            new_elfs = diffs + elf
            mask_arr = ~(elf_pos[:, None] == new_elfs).all(-1).any(0)
            if mask_arr.all():
                continue
            # print(f"{elf}: {[dir for dir in dirs if mask_arr[2*dir: 2*dir + 3].all()]}")
            for dir in dirs:
                if mask_arr[2*dir: 2*dir + 3].all():
                    proposed_moves.append((elf_idx, *(elf + diffs[2*dir + 1])))
                    break
        if not proposed_moves:
            break
        proposed_moves = np.array(proposed_moves)
        unique_pos, unique_inds, unique_counts = np.unique(
            np.array(proposed_moves)[:, 1:],
            return_index=True,
            return_counts=True,
            axis=0,
        )
        update_inds = unique_inds[np.where(unique_counts == 1)]
        elf_pos[proposed_moves[update_inds, 0]] = proposed_moves[update_inds, 1:]
        dirs = np.roll(dirs, -1)
    
    return np.prod(elf_pos.max(0) - elf_pos.min(0) + np.ones(2)) - len(elf_pos), _round + 1


In [4]:
elfs = parse_input("test-input.txt")
task_one(elfs, 10)

100%|██████████| 10/10 [00:00<00:00, 843.69it/s]


(110.0, 10)

In [5]:
elfs = parse_input("input.txt")
task_one(elfs, 10)

100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


(4288.0, 10)

In [6]:
elfs = parse_input("test-input.txt")
task_one(elfs, 10000000)

  0%|          | 19/10000000 [00:00<2:26:10, 1140.13it/s]


(146.0, 20)